<a href="https://colab.research.google.com/github/kyle-gao/PointCloud_RoadSeg_Test/blob/main/Toronto3D_On_Open3dML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/intel-isl/Open3D-ML/master/ml3d/configs/randlanet_toronto3d.yml

--2021-06-24 19:41:10--  https://raw.githubusercontent.com/intel-isl/Open3D-ML/master/ml3d/configs/randlanet_toronto3d.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1370 (1.3K) [text/plain]
Saving to: ‘randlanet_toronto3d.yml’

randlanet_toronto3d 100%[===================>]   1.34K  --.-KB/s    in 0s      

2021-06-24 19:41:10 (47.3 MB/s) - ‘randlanet_toronto3d.yml’ saved [1370/1370]



In [2]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.3MB/s eta 0:04:16tcmalloc: large alloc 1147494400 bytes == 0x5576b7a5a000 @  0x7f24af7bd615 0x55767ebddcdc 0x55767ecbd52a 0x55767ebe0afd 0x55767ecd1fed 0x55767ec54988 0x55767ec4f4ae 0x55767ebe23ea 0x55767ec547f0 0x55767ec4f4ae 0x55767ebe23ea 0x55767ec5132a 0x55767ecd2e36 0x55767ec50853 0x55767ecd2e36 0x55767ec50853 0x55767ecd2e36 0x55767ec50853 0x55767ecd2e36 0x55767ed553e1 0x55767ecb56a9 0x55767ec20cc4 0x55767ebe1559 0x55767ec554f8 0x55767ebe230a 0x55767ec503b5 0x55767ec4f7ad 0x55767ebe23ea 0x55767ec503b5 0x55767ebe230a 0x55767ec503b5
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:22tcmalloc: large alloc 1434370048 bytes == 0x5576fc0b0000 @  0x7f24af7bd615 0x55767ebddcdc 0x55767ecbd52a 0x55767ebe0afd 0x55767ecd1fed 0x55767ec54988 0x55767ec4f4ae 0x55767ebe23ea 0x55767ec547f0 0x55767ec4f4ae 0x55767ebe23ea 0x55767ec5132a 0x55767ecd2e36 0x55767ec50853 0

In [3]:
!pip install open3d

     |████████████████████████████████| 300.6MB 49kB/s 
     |████████████████████████████████| 1.0MB 30.2MB/s 
     |████████████████████████████████| 645kB 33.3MB/s 
     |████████████████████████████████| 8.2MB 35.6MB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
     |████████████████████████████████| 430kB 34.2MB/s 
     |████████████████████████████████| 399kB 43.7MB/s 
     |████████████████████████████████| 61kB 1.1MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: jupyter-server 1.9.0 has requirement jupyter-client>=6.1.1, but you'll have

In [4]:
import torch
import shutil
import open3d.ml.torch as ml3d
import pprint
import open3d.ml as _ml3d


--------------------------------------------------------------------------------

 Using the Open3D PyTorch ops with CUDA 11 may have stability issues!

 We recommend to compile PyTorch from source with compile flags
   '-Xcompiler -fno-gnu-unique'

 or use the PyTorch wheels at
   https://github.com/intel-isl/open3d_downloads/releases/tag/torch1.7.1


 Ignore this message if PyTorch has been compiled with the aforementioned
 flags.

 See https://github.com/intel-isl/Open3D/issues/3324 and
 https://github.com/pytorch/pytorch/issues/52663 for more information on this
 problem.

--------------------------------------------------------------------------------



In [5]:
!gdown --id 1K-46fRl2cDaZeZPVEU6zquSj8fgjae6u

Downloading...
From: https://drive.google.com/uc?id=1K-46fRl2cDaZeZPVEU6zquSj8fgjae6u
To: /content/Toronto_3D.zip
1.14GB [00:08, 131MB/s] 


In [6]:
framework = "torch" # or tf
cfg_file ="/content/randlanet_toronto3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

In [7]:
shutil.unpack_archive("/content/Toronto_3D.zip","/content/Data/")

In [8]:
Pipeline = _ml3d.utils.get_module("pipeline", cfg.pipeline.name, framework)
Model = _ml3d.utils.get_module("model", cfg.model.name, framework)
Dataset = _ml3d.utils.get_module("dataset", cfg.dataset.name)

In [9]:
cfg.dataset['dataset_path'] = "/content/Data/Toronto_3D"
dataset = Dataset(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
model = Model(**cfg.model)
pipeline = Pipeline(model, dataset, **cfg.pipeline)
pipeline.cfg.cfg_dict['max_epoch'] = 1

In [10]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": "Toronto3D",
    "model": "RandLaNet",
    "pipeline": "Default Pipeline 1 epoch",
}

In [11]:
#pipeline.run_train()